In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-23 22:12:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  5.67MB/s    in 0.2s    

2022-01-23 22:12:53 (5.67 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
# Create the customers_table DataFrame
# customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
#customers_df.show()

In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0715000322|Can-Am 715000322 ...|
|0984527281|Word Teasers Lear...|
|0991557557|PicoPen - Stainle...|
|1844257517|Yamaha FZ6 Fazer ...|
|1888000104|JLC Clarity Style...|
|1940825113|War Gaming - Stee...|
|3228963691|Zzzz... Sleeping ...|
|3456495633|Motorcycle First ...|
|3696387606|JLC 9005 CREE LED...|
|3696387622|JLC CREE 5W High ...|
|4678615123|SEM Paints (SEM15...|
|4678615173|Color Coat Camel ...|
|477961421X|Subaru Impreza No...|
|4891071478|Subaru Impreza GC...|
|4891075627|Toyota Mark II/Ch...|
|5888000272|Elite Mailers Nis...|
|5917025029|Nelson Rigg Weath...|
|5926025419|Joe Rocket RS-2 M...|
|5927026230|Nelson Rigg Unise...|
|592702632X|Nelson Rigg Unise...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
#review_id_df.show()

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

### Deliverable 2: Determine Bias of Vine Reviews

In [10]:
# Filter vine_df to filter out reviews with less than 20
filtered_vine_df = vine_df.filter("total_votes >= 20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|     

In [11]:
# Filter based on percentage of helpful votes
from pyspark.sql.functions import col
helpful_df = filtered_vine_df.filter((col("helpful_votes")/col("total_votes"))>= 0.5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [12]:
# Filter based on if review was a part of the Vine program
vine_helpful_df = helpful_df.filter("vine == 'Y'")
vine_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REKN4PLM5174P|          4|          219|        225|   Y|                N|
| RQOG8IIJX7CKB|          4|           21|         27|   Y|                N|
|R33EJ9PSZR13CM|          4|           26|         30|   Y|                N|
| RD2BFV896F321|          5|           16|         20|   Y|                N|
|R37GSXLA11XF8N|          5|           30|         34|   Y|                N|
| RQBQYBM9VMSH4|          5|           19|         20|   Y|                N|
|R2YE4F9ESKFWEK|          3|           20|         23|   Y|                N|
|R1E4L7HHB8TBP9|          4|           80|         92|   Y|                N|
|R2C2WXDXMZL005|          2|           20|         25|   Y|                N|
|R1PPY4U0K03KTE|          5|           31|         38|   Y|     

In [13]:
# Filter based on if review was not part of vine program
nv_helpful_df = helpful_df.filter("vine == 'N'")
nv_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [14]:
# Summary of Vine Reviews
vine_total = vine_helpful_df.count()
vine_five_star = vine_helpful_df.filter("star_rating == 5").count()

# Summary of Non-Vine Reviews
nv_total = nv_helpful_df.count()
nv_five_star = nv_helpful_df.filter("star_rating == 5").count()

print(f'-----------VINE REVIEW SUMMARY-----------')
print(f'The total number of reviews with 20 or more votes: {vine_total}')
print(f'The number of reviews with 5-star ratings that also had 20 or more votes: {vine_five_star}')
print(f'The percentage of 5-star ratings for reviews that had 20 or more votes: {(vine_five_star/vine_total)*100:.2f}%')
print(f'-----------------------------------------')
print(f'---------NON-VINE REVIEW SUMMARY---------')
print(f'The total number of reviews with 20 or more votes: {nv_total}')
print(f'The number of reviews with 5-star ratings that also had 20 or more votes: {nv_five_star}')
print(f'The percentage of 5-star ratings for reviews that had 20 or more votes: {(nv_five_star/nv_total)*100:.2f}%')

-----------VINE REVIEW SUMMARY-----------
The total number of reviews with 20 or more votes: 82
The number of reviews with 5-star ratings that also had 20 or more votes: 33
The percentage of 5-star ratings for reviews that had 20 or more votes: 40.24%
-----------------------------------------
---------NON-VINE REVIEW SUMMARY---------
The total number of reviews with 20 or more votes: 24742
The number of reviews with 5-star ratings that also had 20 or more votes: 12807
The percentage of 5-star ratings for reviews that had 20 or more votes: 51.76%
